In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
def preprocess_image(image):
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Apply Gaussian Blur to smooth the image
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    # Use adaptive thresholding
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                   cv2.THRESH_BINARY_INV, 11, 2)
    # Apply morphological operations to clean up the edges
    kernel = np.ones((5, 5), np.uint8)
    cleaned = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    cleaned = cv2.morphologyEx(cleaned, cv2.MORPH_OPEN, kernel)
    return cleaned

def segment_plants(edges):
    # Find contours in the edge map
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def detect_root_start_points(contours, image):
    root_points = []
    for contour in contours:
        # Filter out small contours that are likely noise
        if cv2.contourArea(contour) < 1800:
            continue
        # Get the bounding box of the contour
        x, y, w, h = cv2.boundingRect(contour)
        # Finding the root start point
        root_point = (x + w // 2, y + h)
        root_points.append(root_point)
        # Draw a circle at the root start point
        cv2.circle(image, root_point, 5, (0, 255, 0), -1)
    return root_points

def process_image(image_path):
    # Load the image
    image = cv2.imread(image_path)
    image = image[400:1000, 750:3500]
    if image is None:
        raise FileNotFoundError(f"Image at path '{image_path}' not found.")
    
    # Preprocess the image
    edges = preprocess_image(image)
    
    # Segment the plants
    contours = segment_plants(edges)
    
    # Detect root start points and annotate the image
    root_points = detect_root_start_points(contours, image)

    root_points = sorted(root_points, key=lambda pt: pt[0])
    
    return image, root_points

def show_image_with_plt(image, root_points):
    # Convert the image from BGR to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Display the image
    plt.imshow(image_rgb)
    plt.title("Detected Root Start Points")
    plt.scatter([pt[0] for pt in root_points], [pt[1] for pt in root_points], color='red')
    plt.show()

def process_folder(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith(".png") or filename.endswith(".jpg") or filename.endswith(".jpeg"):
            image_path = os.path.join(folder_path, filename)
            try:
                image, root_points = process_image(image_path)
                show_image_with_plt(image, root_points)
                print(f"Root Coordinates in order from left to right for {filename}: {root_points}")
            except FileNotFoundError as e:
                print(e)

# Path to the folder containing images
folder_path = 'PNG'

process_folder(folder_path)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'PNG'